In [ ]:
import time
import pennylane as qml
from pennylane import qjit
from pennylane import numpy as np
import matplotlib.pyplot as plt  # Import matplotlib for saving the image

SIZE = 15 # position space
start_time = time.time()
s = SIZE
wires_x = list(range(s)) #[0,1,2,3,4,5]
coin = [s] # coin space

def shift_op():
     for index in range(len(wires_x)-1, 0, -1):
     #    control_values = [1] * index
        qml.ctrl(qml.PauliX, control=wires_x[:index], control_values=[1] * index)(wires=wires_x[index])
     qml.PauliX(wires_x[0])


#@qml.qnode(qml.device("qiskit.basicsim", wires=(wires_x + coin)))
@qml.qnode(qml.device("lightning.qubit", wires=(wires_x + coin)))

def circuit(steps, state):
     qml.BasisState(state, wires=wires_x)
     for i in range(steps): 
        qml.H(coin)
        qml.ctrl(shift_op, control=coin)()
        qml.ctrl(qml.adjoint(shift_op), control=coin, control_values=[0])()
     return qml.probs(wires=wires_x)

steps = 100
state = [0] * SIZE
state[0] = 1
state[1] = 1
state[2] = 0
print(circuit(steps, state))
#print(qml.draw(circuit)(steps=1, state=state))
fig, ax = qml.draw_mpl(circuit, style="pennylane")(steps, state)
fig.savefig("q_walk_circuit.png", dpi=300)
plt.close(fig)
print(fig)

[0.         0.         0.         ... 0.         0.         0.00631336]
Figure(330300x1700)
